# Economic Indicators Analysis: Labour Cost, Inflation, Unemployment, and GDP
## Quarterly Year-over-Year Percentage Changes

### This notebook processes and analyzes four key economic indicators across European countries:
- **LCI**: [Labour Cost Index](https://ec.europa.eu/eurostat/databrowser/view/lc_lci_r2_q/default/table?lang=en)
- **HICP**: [Harmonised Index of Consumer Prices](https://ec.europa.eu/eurostat/databrowser/view/EI_CPHI_M/default/table?lang=en) (Inflation)
- **Unemployment**: [Unemployment rate](https://ec.europa.eu/eurostat/databrowser/view/une_rt_m/default/table?lang=en) changes
- **GDP**: [Gross Domestic Product](https://ec.europa.eu/eurostat/databrowser/view/namq_10_gdp__custom_17409896/default/table?lang=en) growth

### 1. Import Required Libraries

In [1]:
import numpy as np
import openpyxl
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import plotly.subplots as sp
import pycountry 

### 2. Labour Cost Index (LCI) Processing

#### 2.1 Load and Filter LCI Data

In [2]:
# Load Labour Cost Index data
LCI = pd.read_csv("../data/raw/LCI_data.csv")

# Apply filters to get relevant data:
# - Seasonally and calendar adjusted data only
# - Year-over-year percentage changes
# - Labour cost for LCI (compensation of employees plus taxes minus subsidies)
LCI = LCI[(LCI["s_adj"] == "Seasonally and calendar adjusted data") & 
          (LCI["unit"] == "Percentage change compared to same period in previous year") &
          (LCI["lcstruct"] == "Labour cost for LCI (compensation of employees plus taxes minus subsidies)")]

# Remove EU aggregate data (keep only individual countries)
LCI = LCI[~LCI["geo"].str.startswith("Eu")]

# Sort by country for better organization
LCI.sort_values(by="geo", inplace=True)

#### 2.3 Standardize LCI Data Format

In [3]:
# Convert time period to quarter end dates
LCI['quarter_end'] = pd.PeriodIndex(LCI['TIME_PERIOD'], freq='Q').to_timestamp() + pd.offsets.QuarterEnd()

# Keep only necessary columns and rename for consistency
LCI = LCI[['geo', 'quarter_end', 'OBS_VALUE']].drop_duplicates()  # Add drop_duplicates to remove any duplicates
LCI.rename(columns={
    'geo': 'country', 
    'quarter_end': 'time', 
    'OBS_VALUE': 'LCI_pct_change'
}, inplace=True)

# Reset index for clean data
LCI.reset_index(drop=True, inplace=True)

#Save the LCI data to processed
LCI.to_csv('../data/processed/LCI_processed.csv', index=False)

### 3. Harmonised Index of Consumer Prices (HICP) Processing

In [5]:
# fetch the harmonised index of consumer prices (HICP): 
HICP = pd.read_csv("../data/raw/hicp.csv")

# Keep relevant columns and rename for consistency
HICP = HICP[['TIME_PERIOD', 'OBS_VALUE', 'geo']]
HICP.rename(columns={
    'TIME_PERIOD': 'time', 
    'OBS_VALUE': 'hicp_index', 
    'geo': 'country'
}, inplace=True)

# Convert time to datetime
HICP['time'] = pd.to_datetime(HICP['time'], format='%Y-%m')

# Resample to quarterly frequency (last month of quarter)
HICP = HICP.set_index('time').groupby('country')['hicp_index'].resample('Q', label='right').last().reset_index()

# Calculate year-over-year percentage change
HICP['hicp_pct_change'] = HICP.groupby('country')['hicp_index'].pct_change(periods=4) * 100
HICP.dropna(inplace=True)

# Keep only necessary columns
HICP.drop(columns=['hicp_index'], inplace=True)
#save the unemployment data as _processed
HICP.to_csv('../data/processed/hicp_processed.csv', index=False)

### 4. Unemployment Data Processing

In [6]:
# Load unemployment data
UNEMPLOYMENT = pd.read_csv('../data/raw/unemployment.csv')

# Keep relevant columns and rename for consistency
UNEMPLOYMENT = UNEMPLOYMENT[['TIME_PERIOD', 'OBS_VALUE', 'geo']]
UNEMPLOYMENT.rename(columns={
    'TIME_PERIOD': 'time', 
    'OBS_VALUE': 'Unemployment_index', 
    'geo': 'country'
}, inplace=True)

# Convert time to datetime
UNEMPLOYMENT['time'] = pd.to_datetime(UNEMPLOYMENT['time'], format='%Y-%m')

# Resample to quarterly frequency (last month of quarter)
UNEMPLOYMENT = UNEMPLOYMENT.set_index('time').groupby('country')['Unemployment_index'].resample('Q', label='right').last().reset_index()

# Calculate year-over-year percentage change
UNEMPLOYMENT['u_pct_change'] = UNEMPLOYMENT.groupby('country')['Unemployment_index'].pct_change(periods=4) * 100
UNEMPLOYMENT.dropna(inplace=True)

# Keep only necessary columns
UNEMPLOYMENT.drop(columns=['Unemployment_index'], inplace=True)
#save the unemployment data as _processed
UNEMPLOYMENT.to_csv('../data/processed/unemployment_processed.csv', index=False)

### 5. GDP Data Processing

In [7]:
# Load GDP data
GDP = pd.read_csv('../data/raw/gdp.csv')

# Keep relevant columns and rename
GDP = GDP[['TIME_PERIOD', 'OBS_VALUE', 'geo']]
GDP = GDP.rename(columns={
    'geo': 'country', 
    'OBS_VALUE': 'GDP', 
    'TIME_PERIOD': 'time'
})

# Calculate year-over-year percentage change
GDP['gdp_pct_change'] = GDP.groupby('country')['GDP'].pct_change(periods=4) * 100
GDP.dropna(inplace=True)

# Convert to quarter end dates
GDP['quarter_start'] = pd.PeriodIndex(GDP['time'], freq='Q').to_timestamp()
GDP['quarter_end'] = pd.PeriodIndex(GDP['time'], freq='Q').to_timestamp() + pd.offsets.QuarterEnd()

# Keep only necessary columns
GDP = GDP[['country', 'quarter_end', 'gdp_pct_change']]
GDP.rename(columns={
    'quarter_end': 'time', 
    'gdp_pct_change': 'gdp_pct_change_yoy'
}, inplace=True)
#Save the GDP data 
GDP.to_csv('../data/processed/GDP_processed.csv', index=False)

### 6. Merge All Datasets

#### 6.1 Combine All Economic Indicators

In [8]:
# First merge GDP and Unemployment data
GDP_AND_UNEMPLOYMENT = pd.merge(GDP, UNEMPLOYMENT, on=['country', 'time'], how='inner')

# Then merge HICP and LCI data
HICP_LCI = pd.merge(HICP, LCI, on=['country', 'time'], how='inner')

# Finally merge all datasets together
FINAL_DATA = pd.merge(GDP_AND_UNEMPLOYMENT, HICP_LCI, on=['country', 'time'], how='inner')

#This merge would genenerate a lot of duplicated rows, so we drop the rows where time and country are the same: 
# Drop duplicate rows based on time and country columns
FINAL_DATA = FINAL_DATA.drop_duplicates(subset=['time', 'country'])


### 7. Data Quality Verification

In [9]:
# Verify data consistency
print("\n=== DATA QUALITY CHECKS ===")

# Check countries present in final dataset
common_countries = FINAL_DATA['country'].unique()
print(f"\nCountries in final dataset ({len(common_countries)}):")
print(sorted(common_countries))

# Check date coverage by country
date_coverage = FINAL_DATA.groupby('country')['time'].agg(['min', 'max', 'count'])
print("\nDate coverage by country:")
print(date_coverage)

# Summary statistics for each indicator
print("\nSummary statistics for economic indicators:")
summary_stats = FINAL_DATA[['gdp_pct_change_yoy', 'u_pct_change', 'hicp_pct_change', 'LCI_pct_change']].describe()
display(summary_stats.style.format("{:.2f}").set_caption("Summary Statistics"))


=== DATA QUALITY CHECKS ===

Countries in final dataset (31):
['Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus', 'Czechia', 'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Iceland', 'Ireland', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Malta', 'Netherlands', 'Norway', 'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'Spain', 'Sweden', 'Türkiye', 'United Kingdom']

Date coverage by country:
                      min        max  count
country                                    
Austria        2002-03-31 2025-03-31     93
Belgium        2002-03-31 2025-03-31     93
Bulgaria       2002-03-31 2025-03-31     93
Croatia        2009-03-31 2025-03-31     65
Cyprus         2002-03-31 2025-03-31     93
Czechia        2002-03-31 2025-03-31     93
Denmark        2002-03-31 2025-03-31     93
Estonia        2002-03-31 2025-03-31     93
Finland        2002-03-31 2025-03-31     93
France         2002-03-31 2025-03-31     93
Germany        2002-03-31 2025-03

,gdp_pct_change_yoy,u_pct_change,hicp_pct_change,LCI_pct_change
count,2785.00,2785.00,2785.00,2741.00
mean,2.31,1.08,3.23,5.47
std,4.26,20.69,5.27,9.51
min,-21.45,-38.46,-3.97,-20.80
25%,0.57,-10.94,1.02,1.70
50%,2.29,-2.48,2.16,3.80
75%,4.24,8.35,3.73,7.70
max,28.07,213.79,83.39,122.50


### 8. Export Final Dataset

In [10]:
# Save the final merged dataset
FINAL_DATA.to_csv('../data/processed/economic_indicators_quarterly_yoy.csv', index=False)
print("\nFinal dataset saved as 'economic_indicators_quarterly_yoy.csv'")


Final dataset saved as 'economic_indicators_quarterly_yoy.csv'


In [15]:
# Create time series plots for each country showing all economic indicators
unique_countries = FINAL_DATA['country'].unique()

for i, country in enumerate(unique_countries):
    # Filter and sort data for current country
    country_data = FINAL_DATA[FINAL_DATA['country'] == country].copy()
    country_data = country_data.sort_values('time')
    
    # Convert time to datetime if not already
    country_data['time'] = pd.to_datetime(country_data['time'])
    
    # Create figure
    fig = go.Figure()
    
    # Add traces
    fig.add_trace(go.Scatter(
        x=country_data['time'].dt.strftime('%Y-%m-%d'), # Convert to string format
        y=country_data['gdp_pct_change_yoy'],
        name='GDP'
    ))
    
    fig.add_trace(go.Scatter(
        x=country_data['time'].dt.strftime('%Y-%m-%d'),
        y=country_data['u_pct_change'],
        name='Unemployment'
    ))
    
    fig.add_trace(go.Scatter(
        x=country_data['time'].dt.strftime('%Y-%m-%d'),
        y=country_data['hicp_pct_change'],
        name='HICP'
    ))
    
    fig.add_trace(go.Scatter(
        x=country_data['time'].dt.strftime('%Y-%m-%d'),
        y=country_data['LCI_pct_change'],
        name='LCI'
    ))

    # Update traces with colors and widths
    fig.update_traces(selector={'name': 'GDP'}, line={'color': 'blue', 'width': 2})
    fig.update_traces(selector={'name': 'Unemployment'}, line={'color': 'red', 'width': 2})
    fig.update_traces(selector={'name': 'HICP'}, line={'color': 'green', 'width': 2})
    fig.update_traces(selector={'name': 'LCI'}, line={'color': 'purple', 'width': 2})

    # Update layout
    fig.update_layout(
        title=f'Economic Indicators - {country}',
        height=600,
        width=1000,
        xaxis_title="Time Period",
        yaxis_title="Percentage Change",
        hovermode='x unified',
        showlegend=True
    )
    
    # Update x-axis to show years properly
    fig.update_xaxes(
        type='category', # Force x-axis to treat dates as categories
        tickangle=45,
        tickmode='auto',
        nticks=20
    )
    
    # Save figure to PNG file in data directory
    try:
        fig.write_image(f'../data/EDA_full_output/economic_indicators_{country.lower()}.png')
        print(f"Saved visualization for {country}")
    except ValueError as e:
        if "kaleido" in str(e).lower():
            print(f"Kaleido package not available for {country}. Installing...")
            import subprocess
            import sys
            try:
                subprocess.check_call([sys.executable, "-m", "pip", "install", "kaleido"])
                # Try saving again after installation
                fig.write_image(f'../data/EDA_full_output/economic_indicators_{country.lower()}.png')
                print(f"Saved visualization for {country} after installing kaleido")
            except Exception as install_error:
                print(f"Could not install kaleido for {country}: {install_error}")
                print(f"Showing plot for {country} instead of saving")
                fig.show()
        else:
            print(f"Could not save image for {country}: {e}")
            print(f"Showing plot for {country} instead")
            fig.show()
    except Exception as e:
        print(f"Could not save image for {country}: {e}")
        print(f"Showing plot for {country} instead")
        fig.show()

Kaleido package not available for Austria. Installing...
Looking in indexes: https://nexus.turintech.ai/repository/pypi-all/simple, https://nexus.turintech.ai/repository/pypi-all/, https://nexus.turintech.ai/repository/pypi-all/simple/, https://nexus.turintech.ai/repository/pypi-internal/, https://nexus.turintech.ai/repository/pypi-internal/simple, https://nexus.turintech.ai/repository/pypi-external/, https://nexus.turintech.ai/repository/pypi-external/simple, https://nexus.turintech.ai/repository/pypi-snapshot/, https://nexus.turintech.ai/repository/pypi-snapshot/simple/
Could not install kaleido for Austria: 
Image export using the "kaleido" engine requires the Kaleido package,
which can be installed using pip:

    $ pip install --upgrade kaleido

Showing plot for Austria instead of saving


Kaleido package not available for Belgium. Installing...
Looking in indexes: https://nexus.turintech.ai/repository/pypi-all/simple, https://nexus.turintech.ai/repository/pypi-all/, https://nexus.turintech.ai/repository/pypi-all/simple/, https://nexus.turintech.ai/repository/pypi-internal/, https://nexus.turintech.ai/repository/pypi-internal/simple, https://nexus.turintech.ai/repository/pypi-external/, https://nexus.turintech.ai/repository/pypi-external/simple, https://nexus.turintech.ai/repository/pypi-snapshot/, https://nexus.turintech.ai/repository/pypi-snapshot/simple/
Could not install kaleido for Belgium: 
Image export using the "kaleido" engine requires the Kaleido package,
which can be installed using pip:

    $ pip install --upgrade kaleido

Showing plot for Belgium instead of saving


Kaleido package not available for Bulgaria. Installing...
Looking in indexes: https://nexus.turintech.ai/repository/pypi-all/simple, https://nexus.turintech.ai/repository/pypi-all/, https://nexus.turintech.ai/repository/pypi-all/simple/, https://nexus.turintech.ai/repository/pypi-internal/, https://nexus.turintech.ai/repository/pypi-internal/simple, https://nexus.turintech.ai/repository/pypi-external/, https://nexus.turintech.ai/repository/pypi-external/simple, https://nexus.turintech.ai/repository/pypi-snapshot/, https://nexus.turintech.ai/repository/pypi-snapshot/simple/
Could not install kaleido for Bulgaria: 
Image export using the "kaleido" engine requires the Kaleido package,
which can be installed using pip:

    $ pip install --upgrade kaleido

Showing plot for Bulgaria instead of saving


Kaleido package not available for Cyprus. Installing...
Looking in indexes: https://nexus.turintech.ai/repository/pypi-all/simple, https://nexus.turintech.ai/repository/pypi-all/, https://nexus.turintech.ai/repository/pypi-all/simple/, https://nexus.turintech.ai/repository/pypi-internal/, https://nexus.turintech.ai/repository/pypi-internal/simple, https://nexus.turintech.ai/repository/pypi-external/, https://nexus.turintech.ai/repository/pypi-external/simple, https://nexus.turintech.ai/repository/pypi-snapshot/, https://nexus.turintech.ai/repository/pypi-snapshot/simple/
Could not install kaleido for Cyprus: 
Image export using the "kaleido" engine requires the Kaleido package,
which can be installed using pip:

    $ pip install --upgrade kaleido

Showing plot for Cyprus instead of saving


Kaleido package not available for Czechia. Installing...
Looking in indexes: https://nexus.turintech.ai/repository/pypi-all/simple, https://nexus.turintech.ai/repository/pypi-all/, https://nexus.turintech.ai/repository/pypi-all/simple/, https://nexus.turintech.ai/repository/pypi-internal/, https://nexus.turintech.ai/repository/pypi-internal/simple, https://nexus.turintech.ai/repository/pypi-external/, https://nexus.turintech.ai/repository/pypi-external/simple, https://nexus.turintech.ai/repository/pypi-snapshot/, https://nexus.turintech.ai/repository/pypi-snapshot/simple/
Could not install kaleido for Czechia: 
Image export using the "kaleido" engine requires the Kaleido package,
which can be installed using pip:

    $ pip install --upgrade kaleido

Showing plot for Czechia instead of saving


...

Kaleido package not available for Sweden. Installing...
Looking in indexes: https://nexus.turintech.ai/repository/pypi-all/simple, https://nexus.turintech.ai/repository/pypi-all/, https://nexus.turintech.ai/repository/pypi-all/simple/, https://nexus.turintech.ai/repository/pypi-internal/, https://nexus.turintech.ai/repository/pypi-internal/simple, https://nexus.turintech.ai/repository/pypi-external/, https://nexus.turintech.ai/repository/pypi-external/simple, https://nexus.turintech.ai/repository/pypi-snapshot/, https://nexus.turintech.ai/repository/pypi-snapshot/simple/
Could not install kaleido for Sweden: 
Image export using the "kaleido" engine requires the Kaleido package,
which can be installed using pip:

    $ pip install --upgrade kaleido

Showing plot for Sweden instead of saving


Kaleido package not available for Slovenia. Installing...
Looking in indexes: https://nexus.turintech.ai/repository/pypi-all/simple, https://nexus.turintech.ai/repository/pypi-all/, https://nexus.turintech.ai/repository/pypi-all/simple/, https://nexus.turintech.ai/repository/pypi-internal/, https://nexus.turintech.ai/repository/pypi-internal/simple, https://nexus.turintech.ai/repository/pypi-external/, https://nexus.turintech.ai/repository/pypi-external/simple, https://nexus.turintech.ai/repository/pypi-snapshot/, https://nexus.turintech.ai/repository/pypi-snapshot/simple/
Could not install kaleido for Slovenia: 
Image export using the "kaleido" engine requires the Kaleido package,
which can be installed using pip:

    $ pip install --upgrade kaleido

Showing plot for Slovenia instead of saving


Kaleido package not available for Slovakia. Installing...
Looking in indexes: https://nexus.turintech.ai/repository/pypi-all/simple, https://nexus.turintech.ai/repository/pypi-all/, https://nexus.turintech.ai/repository/pypi-all/simple/, https://nexus.turintech.ai/repository/pypi-internal/, https://nexus.turintech.ai/repository/pypi-internal/simple, https://nexus.turintech.ai/repository/pypi-external/, https://nexus.turintech.ai/repository/pypi-external/simple, https://nexus.turintech.ai/repository/pypi-snapshot/, https://nexus.turintech.ai/repository/pypi-snapshot/simple/
Could not install kaleido for Slovakia: 
Image export using the "kaleido" engine requires the Kaleido package,
which can be installed using pip:

    $ pip install --upgrade kaleido

Showing plot for Slovakia instead of saving


Kaleido package not available for Türkiye. Installing...
Looking in indexes: https://nexus.turintech.ai/repository/pypi-all/simple, https://nexus.turintech.ai/repository/pypi-all/, https://nexus.turintech.ai/repository/pypi-all/simple/, https://nexus.turintech.ai/repository/pypi-internal/, https://nexus.turintech.ai/repository/pypi-internal/simple, https://nexus.turintech.ai/repository/pypi-external/, https://nexus.turintech.ai/repository/pypi-external/simple, https://nexus.turintech.ai/repository/pypi-snapshot/, https://nexus.turintech.ai/repository/pypi-snapshot/simple/
Could not install kaleido for Türkiye: 
Image export using the "kaleido" engine requires the Kaleido package,
which can be installed using pip:

    $ pip install --upgrade kaleido

Showing plot for Türkiye instead of saving


Kaleido package not available for United Kingdom. Installing...
Looking in indexes: https://nexus.turintech.ai/repository/pypi-all/simple, https://nexus.turintech.ai/repository/pypi-all/, https://nexus.turintech.ai/repository/pypi-all/simple/, https://nexus.turintech.ai/repository/pypi-internal/, https://nexus.turintech.ai/repository/pypi-internal/simple, https://nexus.turintech.ai/repository/pypi-external/, https://nexus.turintech.ai/repository/pypi-external/simple, https://nexus.turintech.ai/repository/pypi-snapshot/, https://nexus.turintech.ai/repository/pypi-snapshot/simple/
Could not install kaleido for United Kingdom: 
Image export using the "kaleido" engine requires the Kaleido package,
which can be installed using pip:

    $ pip install --upgrade kaleido

Showing plot for United Kingdom instead of saving
